In [7]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import h5py
import warnings
warnings.filterwarnings('ignore')

df = pd.DataFrame(np.array(h5py.File("resnet50_dim512.hdf5")['features']))
df1 = pd.DataFrame(np.array(h5py.File("resnet50_dim512.hdf5")['labels']))

frames = [df, df1]

result = pd.concat(frames, axis=1)
result.head()
result.to_csv('meinDatensatz.csv', encoding='utf-8', index=False)

In [34]:
import pandas as pd
df2 = pd.read_csv('meinDatensatz.csv')
df2

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,0.1
0,-0.299539,0.000218,0.126093,-0.105359,0.082559,0.310264,-0.014708,0.167723,-0.110598,0.133278,...,-0.002834,-0.006792,-0.009844,0.003426,0.006448,-0.022776,-0.002778,0.006110,-0.000451,b'airplane'
1,-0.238882,0.207418,0.403696,-0.042641,0.125476,0.005489,-0.211460,0.063775,0.219866,0.002910,...,0.002903,-0.002855,0.006121,-0.003184,0.001160,-0.000673,-0.003996,0.000689,0.000895,b'airplane'
2,-0.246394,0.295782,0.386239,-0.122544,0.370484,0.193419,-0.045755,0.046651,-0.084080,-0.212352,...,0.002664,0.003390,-0.000866,-0.008851,0.017509,0.007605,-0.013395,-0.003331,0.005176,b'airplane'
3,-0.394405,0.087641,0.479693,-0.088456,0.179256,0.071931,0.343440,0.052034,0.161800,-0.056811,...,0.000294,0.011065,-0.005340,0.002229,0.000245,0.002062,0.005523,0.005544,0.001628,b'airplane'
4,-0.313822,0.274214,0.273074,0.008724,-0.002828,-0.000767,-0.127145,-0.021229,-0.121962,-0.161088,...,0.020541,-0.001229,0.007005,-0.003378,-0.006367,-0.004049,0.014879,0.000004,0.007050,b'airplane'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.331840,-0.092449,-0.362815,0.159686,-0.078759,0.132077,0.180625,-0.112588,-0.199294,-0.155109,...,0.005706,0.000723,-0.005620,-0.000006,-0.004052,-0.001661,0.005998,-0.002015,0.003653,b'truck'
996,-0.488419,0.181747,-0.331930,0.437419,0.064786,0.123709,-0.128111,-0.025579,0.032743,0.022254,...,-0.002859,-0.009365,0.004512,-0.001977,-0.021050,-0.002419,0.010845,0.002813,-0.004593,b'truck'
997,-0.454410,-0.172160,-0.173295,0.181851,0.018338,0.217673,0.061258,-0.166044,-0.152239,-0.177218,...,-0.015935,0.009705,-0.022995,-0.010283,0.010981,-0.004319,0.006246,-0.005833,0.001570,b'truck'
998,-0.467925,-0.154821,-0.353700,0.350269,0.041763,0.101077,-0.041603,0.103436,0.004944,-0.116319,...,0.000636,-0.002711,-0.018510,0.001335,0.000206,0.006879,0.004018,0.009150,0.006693,b'truck'


In [26]:
# pytorch mlp for multiclass classification
from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.transforms import Normalize
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
                
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F

# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path)
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

class MapNet(nn.Module):
    def __init__(self, feature_dim, num_classes):
        super(MapNet, self).__init__()
        self.in_features = feature_dim

        # set up the mapping layers (from feature space dimension to feature space dimension)
        self.mapping = torch.nn.Sequential(
            nn.Linear(in_features=self.in_features, out_features=self.in_features),
            nn.ReLU(inplace=False),
            nn.Linear(in_features=self.in_features, out_features=self.in_features),
            nn.ReLU(inplace=False),
            nn.Linear(in_features=self.in_features, out_features=self.in_features),
        )

        self.classification = nn.Linear(in_features=self.in_features, out_features=num_classes)
        

    def forward(self, x):
        x = F.relu(self.mapping(x))
        x = self.classification(x)
        return x
      

# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(500):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # convert to class labels
        yhat = argmax(yhat, axis=1)
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

# prepare the data
path = 'meinDatensatz.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MapNet(512, 10)
# train the model
train_model(train_dl, model)
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)

670 330
Accuracy: 0.739


# NAS-Paper

In [22]:
def get_batch_jacobian(net, x, target, to, device, args=None):
    net.zero_grad()

    x.requires_grad_(True)

    _, y = net(x)

    y.backward(torch.ones_like(y))
    jacob = x.grad.detach()

    return jacob, target.detach()


def eval_score(jacob, labels=None):
    corrs = np.corrcoef(jacob)
    v, _  = np.linalg.eig(corrs)
    k = 1e-5
    return -np.sum(np.log(v + k) + 1./(v + k))

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [30]:
data_iterator = iter(train_dl)
x, target = next(data_iterator)
print(target)
x, target = x.to(device), target.to(device)


model = model.to(device)
    
jacobs, labels= get_batch_jacobian(model, x, target, 1, device)
jacobs = jacobs.reshape(jacobs.size(0), -1).cpu().numpy()

try:
    s = eval_score(jacobs, labels)
except Exception as e:
    print(e)
    s = np.nan

tensor([4, 1, 2, 2, 2, 2, 1, 2, 5, 0, 1, 9, 4, 8, 1, 7, 0, 5, 9, 5, 3, 7, 6, 9,
        4, 8, 8, 0, 1, 4, 2, 2])


ValueError: too many values to unpack (expected 2)